<a href="https://colab.research.google.com/github/Farmhouse121/Financial-Data-Science-in-Python/blob/main/The%20Market's%20Not%20Normal/The_Market's_Not_Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Market's Not Normal
---
## Preamble

Hi, this is the first "walk-through" in my project _Financial Data Science in Python_. You can read about what this project is in my article on Medium [Financial Data Science in Python](https://medium.com/adventures-in-data-science/financial-data-science-in-python-ee66dab460cf).

To execute the code in this notebook, you're going to need to upload the my_library.py file from the associated GitHub repository (because Google will delete it when the virtual machine goes away, and so it's likely no longer available here). This is easy, but nothing will work without it.

Once that is done, execute the next cell...

In [1]:
from my_library import *

22:35:00 Starting...
22:35:00 Installing yfinance into Google notebook...
22:35:08 Installing arch into Google notebook...
22:35:18 Initialized.


## First, Get All of the Data
We're going to load data for the S&P 500 Index from _Yahoo! Finance_ into a dataframe. This is done by the `download()` function of the `yfinance` package that should have already been installed by executing the first cell. (You did that, right?) The ticker for the index is `^GSPC` -- I don't know why.

In [10]:
ticker,begin,end='^GSPC','1928-01-27',today('%Y-%m-%d')
df=download(ticker,begin,end)
df['Daily Return']=df['Adj Close'].pct_change()*hundred
df['Intraday Return']=df['Close']/df['Open']*hundred-hundred
df['Overnight Return']=(hundred+df["Daily Return"])/(hundred+df["Intraday Return"])*hundred-hundred
df.index=pd.DatetimeIndex(df.index).to_period('B')
df.dropna(inplace=True)
df.loc[df["Volume"]==0,"Volume"]=np.nan
display(df)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Daily Return,Intraday Return,Overnight Return
Date,,,,,,,,,
1928-01-30,17.490000,17.490000,17.490000,17.490000,17.490000,NaN,-1.130587,0.000000,-1.130587
1928-01-31,17.570000,17.570000,17.570000,17.570000,17.570000,NaN,0.457404,0.000000,0.457404
1928-02-01,17.530001,17.530001,17.530001,17.530001,17.530001,NaN,-0.227655,0.000000,-0.227655
1928-02-02,17.629999,17.629999,17.629999,17.629999,17.629999,NaN,0.570442,0.000000,0.570442
1928-02-03,17.400000,17.400000,17.400000,17.400000,17.400000,NaN,-1.304592,0.000000,-1.304592
...,...,...,...,...,...,...,...,...,...
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2.513910e+09,0.423169,0.333906,0.088966
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,4781.580078,2.748450e+09,0.143046,0.170315,-0.027222
2023-12-28,4786.439941,4793.299805,4780.979980,4783.350098,4783.350098,2.698860e+09,0.037017,-0.064554,0.101637


In [3]:
today()

'01/03/2024'